In [1]:
import gc

import modin.pandas as pd

import modin
import pandas

In [2]:
df = pd.read_csv("~/data/G1_1e7_1e2_0_0.csv")

To remove this warning, run the following python code before doing dataframe operations:

    import ray
    ray.init()

2023-07-18 17:53:32,604	INFO worker.py:1636 -- Started a local Ray instance.
Please refer to https://modin.readthedocs.io/en/stable/supported_apis/defaulting_to_pandas.html for explanation.
Reason: No such file: '/Users/matthew.powers/Documents/code/my_apps/pydata-examples/modin/~/data/G1_1e7_1e2_0_0.csv'


In [3]:
df

,id1,id2,id3,id4,id5,id6,v1,v2,v3
0,id016,id016,id0000042202,15,24,5971,5,11,37.211254
1,id039,id045,id0000029558,40,49,39457,5,4,48.951141
2,id047,id023,id0000071286,68,20,74463,2,14,60.469241
3,id043,id057,id0000015141,32,43,63743,1,15,7.692145
4,id054,id040,id0000011083,9,25,16920,2,9,22.863525
...,...,...,...,...,...,...,...,...,...
9999995,id003,id044,id0000019084,86,65,81507,4,8,30.926485
9999996,id096,id068,id0000020362,4,85,95388,3,11,53.268709
9999997,id071,id006,id0000046057,52,57,55223,1,13,90.368393
9999998,id057,id099,id0000060922,47,36,90944,3,8,34.878200


In [4]:
df.head(3)

,id1,id2,id3,id4,id5,id6,v1,v2,v3
0,id016,id016,id0000042202,15,24,5971,5,11,37.211254
1,id039,id045,id0000029558,40,49,39457,5,4,48.951141
2,id047,id023,id0000071286,68,20,74463,2,14,60.469241


## 1e8 CSV

In [5]:
%%time

modin.pandas.read_csv("~/data/G1_1e8_1e2_0_0.csv").groupby("id1").agg({"v1": "sum"})

(raylet) Spilled 2053 MiB, 13 objects, write throughput 1307 MiB/s. Set RAY_verbose_spill_logs=0 to disable this message.
(raylet) Spilled 4825 MiB, 17 objects, write throughput 1859 MiB/s.


CPU times: user 1min 5s, sys: 9.46 s, total: 1min 14s
Wall time: 1min 15s


,v1
id1,
id001,2997617
id002,2997194
id003,3001204
id004,3000945
id005,3001731
...,...
id096,2997704
id097,3004476
id098,2996139


In [6]:
%%time

pandas.read_csv("~/data/G1_1e8_1e2_0_0.csv").groupby("id1").agg({"v1": "sum"})

CPU times: user 43.3 s, sys: 6.84 s, total: 50.2 s
Wall time: 49.9 s


,v1
id1,
id001,2997617
id002,2997194
id003,3001204
id004,3000945
id005,3001731
...,...
id096,2997704
id097,3004476
id098,2996139


## 1e8 Parquet

In [7]:
%%time

modin.pandas.read_parquet("~/data/parquet/G1_1e8_1e2_0_0.parquet").groupby("id1").agg(
    {"v1": "sum"}
)

(raylet) Spilled 8536 MiB, 35 objects, write throughput 2156 MiB/s.


CPU times: user 380 ms, sys: 757 ms, total: 1.14 s
Wall time: 24.1 s


,v1
id1,
id001,2997617
id002,2997194
id003,3001204
id004,3000945
id005,3001731
...,...
id096,2997704
id097,3004476
id098,2996139


In [8]:
%%time

pandas.read_parquet("~/data/parquet/G1_1e8_1e2_0_0.parquet").groupby("id1").agg(
    {"v1": "sum"}
)

CPU times: user 22.7 s, sys: 4.05 s, total: 26.7 s
Wall time: 26.4 s


,v1
id1,
id001,2997617
id002,2997194
id003,3001204
id004,3000945
id005,3001731
...,...
id096,2997704
id097,3004476
id098,2996139


## 1e8 Parquet + column pruning

In [9]:
%%time

modin.pandas.read_parquet(
    "~/data/parquet/G1_1e8_1e2_0_0.parquet", columns=["id1", "v1"]
).groupby("id1").agg({"v1": "sum"})

CPU times: user 55.3 ms, sys: 104 ms, total: 160 ms
Wall time: 4.23 s


,v1
id1,
id001,2997617
id002,2997194
id003,3001204
id004,3000945
id005,3001731
...,...
id096,2997704
id097,3004476
id098,2996139


In [10]:
%%time

pandas.read_parquet(
    "~/data/parquet/G1_1e8_1e2_0_0.parquet", columns=["id1", "v1"]
).groupby("id1").agg({"v1": "sum"})

CPU times: user 4.11 s, sys: 932 ms, total: 5.05 s
Wall time: 4.94 s


,v1
id1,
id001,2997617
id002,2997194
id003,3001204
id004,3000945
id005,3001731
...,...
id096,2997704
id097,3004476
id098,2996139


## Does pandas CSV usecols make queries run faster?

In [11]:
%%time

modin.pandas.read_csv("~/data/G1_1e8_1e2_0_0.csv", usecols=["id1", "v1"]).groupby(
    "id1"
).agg({"v1": "sum"})

CPU times: user 19.1 s, sys: 2.88 s, total: 21.9 s
Wall time: 21.1 s


,v1
id1,
id001,2997617
id002,2997194
id003,3001204
id004,3000945
id005,3001731
...,...
id096,2997704
id097,3004476
id098,2996139


In [12]:
%%time

pandas.read_csv("~/data/G1_1e8_1e2_0_0.csv", usecols=["id1", "v1"]).groupby("id1").agg(
    {"v1": "sum"}
)

CPU times: user 18.4 s, sys: 2.88 s, total: 21.3 s
Wall time: 20.9 s


,v1
id1,
id001,2997617
id002,2997194
id003,3001204
id004,3000945
id005,3001731
...,...
id096,2997704
id097,3004476
id098,2996139


## 1e9 CSV + usecols

In [13]:
%%time

modin.pandas.read_csv("~/data/G1_1e9_1e2_0_0.csv", usecols=["id1", "v1"]).groupby(
    "id1"
).agg({"v1": "sum"})

(raylet) Spilled 16684 MiB, 56 objects, write throughput 2696 MiB/s.


CPU times: user 3min 13s, sys: 31.9 s, total: 3min 45s
Wall time: 3min 49s


,v1
id1,
id001,30009448
id002,29996534
id003,30003365
id004,30015990
id005,29993888
...,...
id096,29993372
id097,30015928
id098,29997789


In [14]:
%%time

pandas.read_csv("~/data/G1_1e9_1e2_0_0.csv", usecols=["id1", "v1"]).groupby("id1").agg(
    {"v1": "sum"}
)

CPU times: user 3min 6s, sys: 26.1 s, total: 3min 32s
Wall time: 3min 30s


,v1
id1,
id001,30009448
id002,29996534
id003,30003365
id004,30015990
id005,29993888
...,...
id096,29993372
id097,30015928
id098,29997789


## 1e9 CSV + pyarrow + usecols

In [15]:
%%time

modin.pandas.read_csv(
    "~/data/G1_1e9_1e2_0_0.csv", engine="pyarrow", usecols=["id1", "v1"]
).groupby("id1").agg({"v1": "sum"})

(raylet) Spilled 33475 MiB, 79 objects, write throughput 2495 MiB/s.


CPU times: user 2min 31s, sys: 53.8 s, total: 3min 24s
Wall time: 4min 35s


,v1
id1,
id001,30009448
id002,29996534
id003,30003365
id004,30015990
id005,29993888
...,...
id096,29993372
id097,30015928
id098,29997789


In [ ]:
%%time

pandas.read_csv(
    "~/data/G1_1e9_1e2_0_0.csv", engine="pyarrow", usecols=["id1", "v1"]
).groupby("id1").agg({"v1": "sum"})

## 1e9 Parquet with column pruning

In [3]:
%%time

modin.pandas.read_parquet(
    "~/data/G1_1e9_1e2_0_0.parquet", columns=["id1", "v1"]
).groupby("id1").agg({"v1": "sum"})

(raylet) Spilled 3051 MiB, 5 objects, write throughput 1081 MiB/s. Set RAY_verbose_spill_logs=0 to disable this message.
(raylet) Spilled 5340 MiB, 8 objects, write throughput 1440 MiB/s.


CPU times: user 2.16 s, sys: 772 ms, total: 2.93 s
Wall time: 20.2 s


(raylet) Spilled 9538 MiB, 26 objects, write throughput 1970 MiB/s.


,v1
id1,
id001,30009448
id002,29996534
id003,30003365
id004,30015990
id005,29993888
...,...
id096,29993372
id097,30015928
id098,29997789


In [4]:
%%time

pandas.read_parquet(
    "~/data/G1_1e9_1e2_0_0.parquet", columns=["id1", "v1"]
).groupby("id1").agg({"v1": "sum"})

CPU times: user 45.4 s, sys: 10.8 s, total: 56.2 s
Wall time: 55 s


,v1
id1,
id001,30009448
id002,29996534
id003,30003365
id004,30015990
id005,29993888
...,...
id096,29993372
id097,30015928
id098,29997789


## Print library versions

In [5]:
!conda list

# packages in environment at /Users/matthew.powers/opt/miniconda3/envs/modin:
#
# Name                    Version                   Build  Channel
aiohttp                   3.8.4                    pypi_0    pypi
aiohttp-cors              0.7.0                    pypi_0    pypi
aiosignal                 1.3.1                    pypi_0    pypi
anyio                     3.7.1              pyhd8ed1ab_0    conda-forge
appnope                   0.1.3              pyhd8ed1ab_0    conda-forge
argon2-cffi               21.3.0             pyhd8ed1ab_0    conda-forge
argon2-cffi-bindings      21.2.0           py39h02fc5c5_3    conda-forge
asttokens                 2.2.1              pyhd8ed1ab_0    conda-forge
async-lru                 2.0.3              pyhd8ed1ab_0    conda-forge
async-timeout             4.0.2                    pypi_0    pypi
attrs                     23.1.0             pyh71513ae_1    conda-forge
babel                     2.12.1             pyhd8ed1ab_1    conda-forge
backc